In [1]:
import pandas as pd

In [2]:
dataset = pd.read_csv('Social_Network_Ads.csv')

In [3]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [7]:
dataset = pd.get_dummies(dataset,drop_first = True)

In [9]:
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,True
1,15810944,35,20000,0,True
2,15668575,26,43000,0,False
3,15603246,27,57000,0,False
4,15804002,19,76000,0,True
...,...,...,...,...,...
395,15691863,46,41000,1,False
396,15706071,51,23000,1,True
397,15654296,50,20000,1,False
398,15755018,36,33000,0,True


In [11]:
dataset.columns

Index(['User ID', 'Age', 'EstimatedSalary', 'Purchased', 'Gender_Male'], dtype='object')

In [13]:
dataset['Purchased'].value_counts()

Purchased
0    257
1    143
Name: count, dtype: int64

In [15]:
indep = dataset[['User ID', 'Age', 'EstimatedSalary', 'Gender_Male']]
dep = dataset['Purchased']

In [17]:
indep.shape

(400, 4)

In [19]:
indep

,User ID,Age,EstimatedSalary,Gender_Male
0,15624510,19,19000,True
1,15810944,35,20000,True
2,15668575,26,43000,False
3,15603246,27,57000,False
4,15804002,19,76000,True
...,...,...,...,...
395,15691863,46,41000,False
396,15706071,51,23000,True
397,15654296,50,20000,False
398,15755018,36,33000,True


In [21]:
dep

0      0
1      0
2      0
3      0
4      0
      ..
395    1
396    1
397    1
398    0
399    1
Name: Purchased, Length: 400, dtype: int64

In [23]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(indep,dep, test_size = 0.30, random_state = 0)

In [25]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.transform(X_test)

In [27]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
param_grid = {'criterion':['gini','entropy'],
              'max_features': ['auto','sqrt','log2'],
              'splitter':['best','random']}
grid=GridSearchCV(DecisionTreeClassifier(),param_grid,refit=True,verbose=3,cv=5,n_jobs=-1,scoring='f1_weighted')
grid.fit(X_train,Y_train)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


C:\Anaconda 3\Lib\site-packages\sklearn\model_selection\_validation.py:547: FitFailedWarning: 
20 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
9 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Anaconda 3\Lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Anaconda 3\Lib\site-packages\sklearn\base.py", line 1467, in wrapper
    estimator._validate_params()
  File "C:\Anaconda 3\Lib\site-packages\sklearn\base.py", line 666, in _validate_params
    validate_parameter_constraints(
  File "C:\Anaconda 3\Lib\site-packages\sklearn\utils\_param_validation.py", line 95, in validat

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'splitter': ['best', 'random']},
             scoring='f1_weighted', verbose=3)

In [29]:
result=grid.cv_results_
#print(result)

Y_pred = grid.predict(X_test)
#print(y_pred)
   
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(Y_test, Y_pred)
#print(cm)

from sklearn.metrics import classification_report

clf_report = classification_report(Y_test, Y_pred)

#print(clf_report)

In [31]:
from sklearn.metrics import f1_score

f1_macro=f1_score(Y_test,Y_pred,average='weighted')

print("The f1_macro value for best parameter {}:".format(grid.best_params_),f1_macro)

The f1_macro value for best parameter {'criterion': 'gini', 'max_features': 'sqrt', 'splitter': 'best'}: 0.8401354928322458


In [33]:
print("The confusion Matrix:\n",cm)

The confusion Matrix:
 [[71  8]
 [11 30]]


In [35]:
print("The report:\n",clf_report)

The report:
               precision    recall  f1-score   support

           0       0.87      0.90      0.88        79
           1       0.79      0.73      0.76        41

    accuracy                           0.84       120
   macro avg       0.83      0.82      0.82       120
weighted avg       0.84      0.84      0.84       120



In [37]:
#AUC ROC stands for “Area Under the Curve” of the “Receiver Operating Characteristic” curve. 
from sklearn.metrics import roc_auc_score

roc_auc_score(Y_test,grid.predict_proba(X_test)[:,1])

0.8152207471441802

In [39]:
table=pd.DataFrame.from_dict(result)

table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_features,param_splitter,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.000000,0.000000,0.000000,0.000000,gini,auto,best,"{'criterion': 'gini', 'max_features': 'auto', ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,9
1,0.003127,0.006254,0.000000,0.000000,gini,auto,random,"{'criterion': 'gini', 'max_features': 'auto', ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,9
2,0.158808,0.072836,0.059644,0.024477,gini,sqrt,best,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.855314,0.875644,0.807622,0.841653,0.927778,0.861602,0.039819,1
3,0.006799,0.004384,0.004778,0.005477,gini,sqrt,random,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.789152,0.772092,0.754941,0.747222,0.785714,0.769824,0.016514,8
4,0.035259,0.014364,0.012141,0.004124,gini,log2,best,"{'criterion': 'gini', 'max_features': 'log2', ...",0.758813,0.802399,0.807155,0.841045,0.911105,0.824103,0.050750,3
5,0.006164,0.005034,0.019083,0.010989,gini,log2,random,"{'criterion': 'gini', 'max_features': 'log2', ...",0.819142,0.858503,0.789949,0.785714,0.839990,0.818660,0.028116,6
6,0.004347,0.004215,0.000000,0.000000,entropy,auto,best,"{'criterion': 'entropy', 'max_features': 'auto...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,9
7,0.004463,0.002459,0.000000,0.000000,entropy,auto,random,"{'criterion': 'entropy', 'max_features': 'auto...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,9
8,0.009612,0.008357,0.010022,0.002240,entropy,sqrt,best,"{'criterion': 'entropy', 'max_features': 'sqrt...",0.799537,0.841398,0.824293,0.768874,0.890114,0.824843,0.040762,2
9,0.006193,0.001836,0.007834,0.001221,entropy,sqrt,random,"{'criterion': 'entropy', 'max_features': 'sqrt...",0.733523,0.758813,0.821429,0.783333,0.872761,0.793972,0.048918,7
